# Setup

## Get AWS setup

1. From https://boto3.readthedocs.io/en/latest/guide/quickstart.html (Boto is the Amazon Web Services (AWS) SDK for Python)

        sudo apt-get install awscli
        pip install boto3
2. Visit AWS -> IAM -> Add user -> Security Credentials -> Create Access Key
3. Run `aws configure` and enter the ID, code, region (eu-west-1) - ireland, outputformat (blank - leave as JSON)
4. Test with:

        import boto3
        s3 = boto3.resource('s3')
        for b in s3.buckets.all():
            print(b.name)
        
5. From http://distributed.readthedocs.io/en/latest/ec2.html,

        pip install dask-ec2

6. Visit AWS->EC2->Key pairs->Create key pair. I called mine "research". Save the keyfile in .ssh, chmod 600.
7. Get the AMI we want to use (e.g. ubuntu 14.04). Check https://cloud-images.ubuntu.com/locator/ec2/ and search for e.g. `14.04 LTS eu-west-1 hvm ebs`. 

## Running DASK

1. Run `dask-ec2 up --keyname YOUR-AWS-KEY --keypair ~/.ssh/YOUR-AWS-SSH-KEY.pem`. I found I had to also specify the region-name, the ami and tags as the first two have wrong defaults and the tool seems to fail if tags isn't set either.
Also found using ubuntu 16.04 had a SSL wrong version number (see https://github.com/dask/dask-ec2/issues/38 ).
E.g.

        dask-ec2 up --keyname research --keypair .ssh/research.pem --region-name eu-west-1 --ami ami-d37961b5 --tags research:dp

Eventually after a long time, this will finish with:

        Dask.Distributed Installation succeeded

        Addresses
        ---------
        Web Interface:    http://54.229.183.56:8787/status
        TCP Interface:           54.229.183.56:8786

        To connect from the cluster
        ---------------------------

        dask-ec2 ssh  # ssh into head node
        ipython  # start ipython shell

        from dask.distributed import Client, progress
        c = Client('127.0.0.1:8786')  # Connect to scheduler running on the head node

        To connect locally
        ------------------

        Note: this requires you to have identical environments on your local machine and cluster.

        ipython  # start ipython shell

        from dask.distributed import Client, progress
        e = Client('54.229.183.56:8786')  # Connect to scheduler running on the head node

        To destroy
        ----------

        dask-ec2 destroy
        Installing Jupyter notebook on the head node
        DEBUG: Uploading file /tmp/tmpePEm3R to /tmp/.__tmp_copy
        DEBUG: Running command sudo -S bash -c 'cp -rf /tmp/.__tmp_copy /srv/pillar/jupyter.sls' on '54.229.183.56'
        DEBUG: Running command sudo -S bash -c 'rm -rf /tmp/.__tmp_copy' on '54.229.183.56'
        +---------+----------------------+-----------------+
        | Node ID | # Successful actions | # Failed action |
        +=========+======================+=================+
        | node-0  | 17                   | 0               |
        +---------+----------------------+-----------------+
        Jupyter notebook available at http://54.229.183.56:8888/ 

2. Next try; http://jcrist.github.io/dask-sklearn-part-1.html